# Coursework 3 Streaming Algorithm

## **Task1：DGIM**

DGIM is an efficient algorithm in processing large streams. When it's infeasible to store the flowing binary stream, DGIM can estimate the number of 1-bits in the window. In this coding, you're given the stream_data_dgim.txt (binary stream), and you need to implement the DGIM algorithm to count the number of 1-bits. Write code below.

### 1. Set the window size to 1000, and count the number of 1-bits in the current window.

In [5]:
import tracemalloc
N = 1000
tracemalloc.start()
with open('stream_data_dgim.txt',"r") as f:
    bits = f.read()
    bits = bits.split('\t')
    bits = bits[0:len(bits)-1]
print('读入整个文件的内存占用为（单位字节）前一项为当前使用内存，后一项为峰值内存')
print(tracemalloc.get_traced_memory())
tracemalloc.stop()
tracemalloc.start()
class bucket:
    def __init__(self,t):
        self.length = 1
        self.num_one = 1
        self.time_stamp = t
        
def change_num_one(bucket_num_one,num):
    if num in bucket_num_one.keys():
        bucket_num_one[num]+=1
    else:
        bucket_num_one[num]=1
    return bucket_num_one
def bucket_merge(bucket_1,bucket_2,bucket_list,bucket_num_one ):
    if bucket_1.num_one != bucket_2.num_one:
        print('Error')
        print(bucket_num_one)
        return 0
    origin_num_one = bucket_1.num_one
    if bucket_1.time_stamp>bucket_2.time_stamp:
        new_time = bucket_1.time_stamp
        bucket_1.length += bucket_2.length
        bucket_1.num_one+=bucket_2.num_one
        index_b2 = bucket_list.index(bucket_2)
        bucket_list.pop(index_b2)
    else:
        new_time = bucket_2.time_stamp
        bucket_2.length += bucket_1.length
        bucket_2.num_one+=bucket_1.num_one
        index_b1 = bucket_list.index(bucket_1)
        bucket_list.pop(index_b1)
    bucket_num_one[origin_num_one]-=2
    bucket_num_one = change_num_one(bucket_num_one,2*origin_num_one)
    return bucket_list, bucket_num_one

bucket_list = []
bucket_num_one = {}
print('DGIM的内存占用为（单位字节）')
for t in range(len(bits)):
    bit = int(bits[t])
    if len(bucket_list) == 0 or len(bucket_list) == 1 :
        pass
    elif t-bucket_list[0].time_stamp>N:
        bucket_num_one[bucket_list[0].num_one]-=1
        bucket_list.pop(0)
    if bit == 0:
        continue
    new_bucket = bucket(t)
    bucket_list.append(new_bucket)
    bucket_num_one=change_num_one(bucket_num_one,1)
    if bucket_num_one[1]>2:
        flag = 1
    else:
        flag = 0
    trac = len(bucket_list)-1
    while(flag == 1):
        if trac < 2:
            break
        bucket_list, bucket_num_one = bucket_merge(bucket_list[trac-2],bucket_list[trac-1],bucket_list, bucket_num_one)
        trac-=2
        #print(bucket_num_one[bucket_list[trac].num_one])
        if bucket_num_one[bucket_list[trac].num_one] < 3:
            flag = 0
    #桶的合并

print(tracemalloc.get_traced_memory())
tracemalloc.stop()
print('Done')
print('bucket情况为')
print(bucket_num_one)
print(len(bucket_list))
    
    

    

读入整个文件的内存占用为（单位字节）前一项为当前使用内存，后一项为峰值内存
(473223, 672899)
DGIM的内存占用为（单位字节）
(159443, 170456)
Done
bucket情况为
{1: 2, 2: 1, 4: 2, 8: 2, 16: 2, 32: 2, 64: 2, 128: 1, 256: 1}
15


### 代码说明
#### 定义了一个bucket类，包含3个信息（1的个数，维护长度和时间戳）。字典bucket_num_one统计了对于包含某一数量1的bucket有几个。bucket_merge函数实现了两个bucket的合并。函数change_num_one是辅助函数。
#### For循环中是算法的主要实现，包括bucket新建，合并，整个bucket序列的维护

### 2. With the window size 1000, count the number of 1-bits in the last 500 and 200 bits of the bitstream.

In [6]:
stop_time = len(bits)-1
tracemalloc.start()
def count_last_n(bucket_list,n,stop_time):
    count = 0
    for bucket in bucket_list:
        if stop_time-bucket.time_stamp<n:
            if count == 0:
                count+=int(bucket.num_one/2)
            else:
                count+=bucket.num_one
    return count
num_500_one = count_last_n(bucket_list,500,stop_time)
num_200_one = count_last_n(bucket_list,200,stop_time)
print('内存占用为')
print(tracemalloc.get_traced_memory())
tracemalloc.stop()
print(num_500_one)
print(num_200_one)

内存占用为
(3906, 15753)
220
76


### 3. Write a function that accurately counts the number of 1-bits in the current window. Caculate the accuracy of your own DGIM algorithm and compare the running time difference.

In [7]:
from collections import Counter
import time
tracemalloc.start()
def accurate_counting(bits, t, window_size):
    start_time = max(0,t-window_size)
    bit_range = Counter(bits[start_time:t+1])
    return int(bit_range['1'])
bucket_list = []
bucket_num_one = {}
tic = time.time()
for t in range(len(bits)):
    bit = int(bits[t])
    if len(bucket_list) == 0 or len(bucket_list) == 1 :
        pass
    elif t-bucket_list[0].time_stamp>N:
        bucket_num_one[bucket_list[0].num_one]-=1
        bucket_list.pop(0)
    if bit == 0:
        continue
    new_bucket = bucket(t)
    bucket_list.append(new_bucket)
    bucket_num_one=change_num_one(bucket_num_one,1)
    if bucket_num_one[1]>2:
        flag = 1
    else:
        flag = 0
    trac = len(bucket_list)-1
    while(flag == 1):
        if trac < 2:
            break
        bucket_list, bucket_num_one = bucket_merge(bucket_list[trac-2],bucket_list[trac-1],bucket_list, bucket_num_one)
        trac-=2
        #print(bucket_num_one[bucket_list[trac].num_one])
        if bucket_num_one[bucket_list[trac].num_one] < 3:
            flag = 0
    sum_bucket = 0
    for i,buckets in enumerate(bucket_list):
        if t>N:
            if i == 0:
                sum_bucket+= int(buckets.num_one/2)
            else:
                sum_bucket+= buckets.num_one
        else:
            sum_bucket+=buckets.num_one
    toc = time.time()
    total = accurate_counting(bits, t, N)
    time2 = time.time()
    print('时间差距为：'+str(toc-tic-(time2-toc)))
    print('总共：'+str(total))
    print('估计：'+str(sum_bucket))
    print('差距为'+str(total-sum_bucket))
print('内存占用为')
print(tracemalloc.get_traced_memory())
tracemalloc.stop()
    

时间差距为：0.0009970664978027344
总共：1
估计：1
差距为0
时间差距为：0.0009970664978027344
总共：2
估计：2
差距为0
时间差距为：0.0009970664978027344
总共：3
估计：3
差距为0
时间差距为：0.0009970664978027344
总共：4
估计：4
差距为0
时间差距为：0.0009970664978027344
总共：5
估计：5
差距为0
时间差距为：0.0009970664978027344
总共：6
估计：6
差距为0
时间差距为：0.0009970664978027344
总共：7
估计：7
差距为0
时间差距为：0.0009970664978027344
总共：8
估计：8
差距为0
时间差距为：0.0009970664978027344
总共：9
估计：9
差距为0
时间差距为：0.0009970664978027344
总共：10
估计：10
差距为0
时间差距为：0.0019974708557128906
总共：11
估计：11
差距为0
时间差距为：0.0019974708557128906
总共：12
估计：12
差距为0
时间差距为：0.0019974708557128906
总共：13
估计：13
差距为0
时间差距为：0.0019974708557128906
总共：14
估计：14
差距为0
时间差距为：0.0019974708557128906
总共：15
估计：15
差距为0
时间差距为：0.0019974708557128906
总共：16
估计：16
差距为0
时间差距为：0.0019974708557128906
总共：17
估计：17
差距为0
时间差距为：0.0019974708557128906
总共：18
估计：18
差距为0
时间差距为：0.0019974708557128906
总共：19
估计：19
差距为0
时间差距为：0.0019974708557128906
总共：20
估计：20
差距为0
时间差距为：0.0019974708557128906
总共：21
估计：21
差距为0
时间差距为：0.0019974708557128906
总共：22
估计：22
差距为0
时间差距为：0.0029973983764648438


总共：397
估计：442
差距为-45
时间差距为：0.19499731063842773
总共：398
估计：443
差距为-45
时间差距为：0.19399714469909668
总共：399
估计：444
差距为-45
时间差距为：0.1959974765777588
总共：400
估计：445
差距为-45
时间差距为：0.1959974765777588
总共：401
估计：446
差距为-45
时间差距为：0.1959974765777588
总共：401
估计：383
差距为18
时间差距为：0.1959974765777588
总共：402
估计：384
差距为18
时间差距为：0.19499802589416504
总共：403
估计：385
差距为18
时间差距为：0.19699692726135254
总共：402
估计：386
差距为16
时间差距为：0.19699692726135254
总共：401
估计：387
差距为14
时间差距为：0.19699692726135254
总共：401
估计：388
差距为13
时间差距为：0.19699692726135254
总共：400
估计：389
差距为11
时间差距为：0.19699692726135254
总共：400
估计：390
差距为10
时间差距为：0.1979963779449463
总共：400
估计：391
差距为9
时间差距为：0.1979963779449463
总共：399
估计：392
差距为7
时间差距为：0.1979963779449463
总共：399
估计：393
差距为6
时间差距为：0.1979963779449463
总共：399
估计：394
差距为5
时间差距为：0.1979963779449463
总共：396
估计：395
差距为1
时间差距为：0.1979963779449463
总共：397
估计：396
差距为1
时间差距为：0.19699430465698242
总共：396
估计：397
差距为-1
时间差距为：0.19899845123291016
总共：394
估计：398
差距为-4
时间差距为：0.19899845123291016
总共：395
估计：399
差距为-4
时间差距为：0.203995943069458
总

时间差距为：0.38199710845947266
总共：410
估计：364
差距为46
时间差距为：0.3829972743988037
总共：410
估计：365
差距为45
时间差距为：0.3829972743988037
总共：406
估计：366
差距为40
时间差距为：0.3829972743988037
总共：404
估计：367
差距为37
时间差距为：0.38199806213378906
总共：405
估计：368
差距为37
时间差距为：0.38399648666381836
总共：405
估计：369
差距为36
时间差距为：0.38399648666381836
总共：406
估计：370
差距为36
时间差距为：0.38399648666381836
总共：407
估计：371
差距为36
时间差距为：0.38399648666381836
总共：405
估计：372
差距为33
时间差距为：0.38399648666381836
总共：405
估计：373
差距为32
时间差距为：0.38399648666381836
总共：404
估计：374
差距为30
时间差距为：0.3849971294403076
总共：405
估计：375
差距为30
时间差距为：0.3849971294403076
总共：404
估计：376
差距为28
时间差距为：0.3849971294403076
总共：404
估计：377
差距为27
时间差距为：0.3849971294403076
总共：405
估计：378
差距为27
时间差距为：0.3849971294403076
总共：405
估计：379
差距为26
时间差距为：0.38599681854248047
总共：406
估计：380
差距为26
时间差距为：0.38599681854248047
总共：407
估计：381
差距为26
时间差距为：0.38599681854248047
总共：406
估计：382
差距为24
时间差距为：0.38599681854248047
总共：406
估计：383
差距为23
时间差距为：0.3849954605102539
总共：405
估计：384
差距为21
时间差距为：0.3919966220855713
总共：405
估计：385
差距为2

时间差距为：0.5689976215362549
总共：403
估计：476
差距为-73
时间差距为：0.5699965953826904
总共：404
估计：477
差距为-73
时间差距为：0.5699965953826904
总共：405
估计：478
差距为-73
时间差距为：0.5699965953826904
总共：405
估计：479
差距为-74
时间差距为：0.5709974765777588
总共：404
估计：480
差距为-76
时间差距为：0.5709974765777588
总共：403
估计：481
差距为-78
时间差距为：0.5709974765777588
总共：404
估计：482
差距为-78
时间差距为：0.5709974765777588
总共：403
估计：483
差距为-80
时间差距为：0.5709974765777588
总共：403
估计：484
差距为-81
时间差距为：0.5699963569641113
总共：403
估计：485
差距为-82
时间差距为：0.5719985961914062
总共：403
估计：486
差距为-83
时间差距为：0.5719985961914062
总共：403
估计：487
差距为-84
时间差距为：0.5719985961914062
总共：404
估计：488
差距为-84
时间差距为：0.5719985961914062
总共：404
估计：489
差距为-85
时间差距为：0.5719985961914062
总共：402
估计：490
差距为-88
时间差距为：0.5719985961914062
总共：402
估计：491
差距为-89
时间差距为：0.5729968547821045
总共：402
估计：492
差距为-90
时间差距为：0.5729968547821045
总共：403
估计：493
差距为-90
时间差距为：0.5729968547821045
总共：403
估计：494
差距为-91
时间差距为：0.5729968547821045
总共：401
估计：495
差距为-94
时间差距为：0.5729968547821045
总共：402
估计：496
差距为-94
时间差距为：0.5729968547821045
总共：402
估计

时间差距为：0.7554724216461182
总共：353
估计：334
差距为19
时间差距为：0.7574727535247803
总共：354
估计：335
差距为19
时间差距为：0.7574727535247803
总共：355
估计：336
差距为19
时间差距为：0.7574727535247803
总共：356
估计：337
差距为19
时间差距为：0.7564733028411865
总共：356
估计：338
差距为18
时间差距为：0.758472204208374
总共：356
估计：339
差距为17
时间差距为：0.758472204208374
总共：355
估计：340
差距为15
时间差距为：0.758472204208374
总共：351
估计：341
差距为10
时间差距为：0.758472204208374
总共：352
估计：342
差距为10
时间差距为：0.758472204208374
总共：351
估计：343
差距为8
时间差距为：0.758472204208374
总共：350
估计：344
差距为6
时间差距为：0.7574713230133057
总共：350
估计：345
差距为5
时间差距为：0.7594730854034424
总共：348
估计：346
差距为2
时间差距为：0.7594730854034424
总共：348
估计：347
差距为1
时间差距为：0.7594730854034424
总共：346
估计：348
差距为-2
时间差距为：0.7594730854034424
总共：346
估计：349
差距为-3
时间差距为：0.7594730854034424
总共：347
估计：350
差距为-3
时间差距为：0.7584714889526367
总共：347
估计：351
差距为-4
时间差距为：0.760474681854248
总共：346
估计：352
差距为-6
时间差距为：0.760474681854248
总共：346
估计：353
差距为-7
时间差距为：0.760474681854248
总共：347
估计：354
差距为-7
时间差距为：0.760474681854248
总共：345
估计：355
差距为-10
时间差距为：0.760474681854248


...

总共：404
估计：448
差距为-44
时间差距为：6.0079686641693115
总共：403
估计：449
差距为-46
时间差距为：6.0079686641693115
总共：404
估计：450
差距为-46
时间差距为：6.0079686641693115
总共：405
估计：451
差距为-46
时间差距为：6.006968975067139
总共：405
估计：452
差距为-47
时间差距为：6.008968353271484
总共：406
估计：453
差距为-47
时间差距为：6.008968353271484
总共：406
估计：454
差距为-48
时间差距为：6.008968353271484
总共：406
估计：455
差距为-49
时间差距为：6.008968353271484
总共：406
估计：456
差距为-50
时间差距为：6.008968353271484
总共：406
估计：457
差距为-51
时间差距为：6.007968187332153
总共：406
估计：458
差距为-52
时间差距为：6.009968519210815
总共：406
估计：459
差距为-53
时间差距为：6.009968519210815
总共：407
估计：460
差距为-53
时间差距为：6.009968519210815
总共：407
估计：461
差距为-54
时间差距为：6.009968519210815
总共：408
估计：462
差距为-54
时间差距为：6.009968519210815
总共：407
估计：463
差距为-56
时间差距为：6.0089685916900635
总共：407
估计：464
差距为-57
时间差距为：6.010968446731567
总共：407
估计：465
差距为-58
时间差距为：6.010968446731567
总共：407
估计：466
差距为-59
时间差距为：6.010968446731567
总共：407
估计：467
差距为-60
时间差距为：6.010968446731567
总共：406
估计：468
差距为-62
时间差距为：6.010968446731567
总共：407
估计：469
差距为-62
时间差距为：6.009968996047974
总共：407

差距为-119
时间差距为：6.194970369338989
总共：408
估计：527
差距为-119
时间差距为：6.194970369338989
总共：408
估计：528
差距为-120
时间差距为：6.193970441818237
总共：409
估计：529
差距为-120
时间差距为：6.195970296859741
总共：409
估计：530
差距为-121
时间差距为：6.195970296859741
总共：408
估计：531
差距为-123
时间差距为：6.195970296859741
总共：408
估计：532
差距为-124
时间差距为：6.195970296859741
总共：408
估计：533
差距为-125
时间差距为：6.196969509124756
总共：408
估计：534
差距为-126
时间差距为：6.196969509124756
总共：408
估计：535
差距为-127
时间差距为：6.196969509124756
总共：408
估计：344
差距为64
时间差距为：6.196969509124756
总共：408
估计：345
差距为63
时间差距为：6.196969509124756
总共：409
估计：346
差距为63
时间差距为：6.196969509124756
总共：409
估计：347
差距为62
时间差距为：6.197968244552612
总共：409
估计：348
差距为61
时间差距为：6.197968244552612
总共：410
估计：349
差距为61
时间差距为：6.197968244552612
总共：409
估计：350
差距为59
时间差距为：6.197968244552612
总共：409
估计：351
差距为58
时间差距为：6.197968244552612
总共：405
估计：352
差距为53
时间差距为：6.198968172073364
总共：405
估计：353
差距为52
时间差距为：6.198968172073364
总共：405
估计：354
差距为51
时间差距为：6.198968172073364
总共：406
估计：355
差距为51
时间差距为：6.198968172073364
总共：406
估计：356
差距为50
时间差距为：

估计：431
差距为-31
时间差距为：6.382968425750732
总共：399
估计：432
差距为-33
时间差距为：6.382968425750732
总共：399
估计：433
差距为-34
时间差距为：6.382968425750732
总共：400
估计：434
差距为-34
时间差距为：6.383968353271484
总共：400
估计：435
差距为-35
时间差距为：6.383968353271484
总共：401
估计：436
差距为-35
时间差距为：6.383968353271484
总共：401
估计：437
差距为-36
时间差距为：6.383968353271484
总共：397
估计：438
差距为-41
时间差距为：6.383968353271484
总共：397
估计：439
差距为-42
时间差距为：6.383968353271484
总共：397
估计：440
差距为-43
时间差距为：6.384968519210815
总共：397
估计：441
差距为-44
时间差距为：6.384968519210815
总共：396
估计：442
差距为-46
时间差距为：6.384968519210815
总共：397
估计：443
差距为-46
时间差距为：6.384968519210815
总共：396
估计：444
差距为-48
时间差距为：6.384968519210815
总共：397
估计：445
差距为-48
时间差距为：6.384968519210815
总共：395
估计：446
差距为-51
时间差距为：6.3859686851501465
总共：396
估计：447
差距为-51
时间差距为：6.3859686851501465
总共：395
估计：448
差距为-53
时间差距为：6.3859686851501465
总共：395
估计：449
差距为-54
时间差距为：6.3859686851501465
总共：395
估计：450
差距为-55
时间差距为：6.3859686851501465
总共：395
估计：451
差距为-56
时间差距为：6.3859686851501465
总共：395
估计：452
差距为-57
时间差距为：6.384968996047974
总共：395
估计：4

时间差距为：6.5699684619903564
总共：398
估计：422
差距为-24
时间差距为：6.5699684619903564
总共：397
估计：423
差距为-26
时间差距为：6.5699684619903564
总共：397
估计：424
差距为-27
时间差距为：6.568968772888184
总共：395
估计：425
差距为-30
时间差距为：6.570968151092529
总共：395
估计：426
差距为-31
时间差距为：6.570968151092529
总共：396
估计：427
差距为-31
时间差距为：6.570968151092529
总共：397
估计：428
差距为-31
时间差距为：6.570968151092529
总共：397
估计：429
差距为-32
时间差距为：6.570968151092529
总共：395
估计：430
差距为-35
时间差距为：6.57196831703186
总共：395
估计：431
差距为-36
时间差距为：6.57196831703186
总共：395
估计：432
差距为-37
时间差距为：6.57196831703186
总共：390
估计：433
差距为-43
时间差距为：6.57196831703186
总共：390
估计：434
差距为-44
时间差距为：6.57196831703186
总共：391
估计：435
差距为-44
时间差距为：6.57196831703186
总共：389
估计：436
差距为-47
时间差距为：6.570968151092529
总共：387
估计：437
差距为-50
时间差距为：6.572968482971191
总共：387
估计：438
差距为-51
时间差距为：6.572968482971191
总共：387
估计：439
差距为-52
时间差距为：6.572968482971191
总共：387
估计：440
差距为-53
时间差距为：6.572968482971191
总共：387
估计：441
差距为-54
时间差距为：6.572968482971191
总共：386
估计：442
差距为-56
时间差距为：6.571967840194702
总共：386
估计：443
差距为-57
时间差距为：6.57396

时间差距为：6.7559685707092285
总共：397
估计：345
差距为52
时间差距为：6.757968187332153
总共：394
估计：346
差距为48
时间差距为：6.757968187332153
总共：395
估计：347
差距为48
时间差距为：6.757968187332153
总共：395
估计：348
差距为47
时间差距为：6.756968259811401
总共：394
估计：349
差距为45
时间差距为：6.758968114852905
总共：394
估计：350
差距为44
时间差距为：6.758968114852905
总共：394
估计：351
差距为43
时间差距为：6.758968114852905
总共：395
估计：352
差距为43
时间差距为：6.758968114852905
总共：396
估计：353
差距为43
时间差距为：6.758968114852905
总共：397
估计：354
差距为43
时间差距为：6.758968114852905
总共：398
估计：355
差距为43
时间差距为：6.759968280792236
总共：399
估计：356
差距为43
时间差距为：6.759968280792236
总共：398
估计：357
差距为41
时间差距为：6.759968280792236
总共：399
估计：358
差距为41
时间差距为：6.759968280792236
总共：399
估计：359
差距为40
时间差距为：6.759968280792236
总共：399
估计：360
差距为39
时间差距为：6.759968280792236
总共：399
估计：361
差距为38
时间差距为：6.760968208312988
总共：400
估计：362
差距为38
时间差距为：6.760968208312988
总共：399
估计：363
差距为36
时间差距为：6.760968208312988
总共：399
估计：364
差距为35
时间差距为：6.760968208312988
总共：399
估计：365
差距为34
时间差距为：6.760968208312988
总共：399
估计：366
差距为33
时间差距为：6.760968208312988
总共：400


#### 实现DGIM算法估计当前window中包含的1的个数，可以从输出结果看出估计与实际之间的差距均没有超过50%，符合该算法的理论分析。

## **Task2: Bloom Filter**

A Bloom filter is a space-efficient probabilistic data structure. Here the task is to implement a bloom filter by yourself. 

### Data loading:

From the NLTK (Natural Language ToolKit) library, we import a large list of English dictionary words, commonly used by the very first spell-checking programs in Unix-like operating systems.

In [8]:
import nltk
from nltk.corpus import words
word_list = words.words()

Then we load another dataset from the NLTK Corpora collection: movie_reviews.

The movie reviews are categorized between positive and negative, so we construct a list of words (usually called bag of words) for each category.

In [9]:
from nltk.corpus import movie_reviews

neg_reviews = []
pos_reviews = []

for fileid in movie_reviews.fileids('neg'):
    neg_reviews.extend(movie_reviews.words(fileid))
for fileid in movie_reviews.fileids('pos'):
    pos_reviews.extend(movie_reviews.words(fileid))

Here we get a data stream (word_list) and 2 query lists (neg_reviews and pos_reviews).

### 1. Write a function that accurately determines whether each word in neg_reviews and pos_reviews belongs to word_list.

In [10]:
def accurate_determine (compare, base):
    list_judge = []
    for c_word in compare:
        if c_word in base:
            list_judge.append(1)
        else:
            list_judge.append(0)
    return list_judge


#list_positive = accurate_determine(pos_reviews,word_list )
#list_negative = accurate_determine(neg_reviews,word_list)
#print(list_positive)
#print(list_negative)
        

 ### 2. Implement the bloom filter by yourself and add all words in word_list in your bloom filter. Compare the running time difference between linear search on a list and multiple hash computations in a Bloom filter.

In [11]:
import numpy as np
def tic():
    global _begin_time
    _begin_time = time.perf_counter()


def toc():
    return 1000 * (time.perf_counter() - _begin_time)
tracemalloc.start()
m = 10*len(word_list)
def hash_1(string, m ,p):
    length = len(string)
    temp_hash_vev = np.zeros(length)
    for i in range(length):
        chara = string[i]
        idx = ord(chara)-ord('a')+1
        if i == 0:
            temp_hash_vev[i] = idx
        else:
            temp_hash_vev[i] = (temp_hash_vev[i-1]*p+idx)%m
    hash_value = int(temp_hash_vev[-1])
    return hash_value
def hash_determine(compare,m,p_list,B):
    list_judge = []
    for string in compare:
        flag = 1
        for p in p_list:
            hash_value = hash_1(string, m ,p)
            if B[hash_value] != 1:
                flag = 0
                break
        list_judge.append(flag)
    return list_judge
                
B = np.zeros(m)
k = 3
p_list = [23,29,41,43,51,57,13,17,19,31,37,19,73,97,83,61]
for word in word_list:
    for p in p_list[0:k]:
        hash_value = hash_1(word,m,p)
        B[hash_value] = 1
tic()
accurate_determine(pos_reviews[0:300],word_list)
print("Linear Search: %.2f ms" % toc())
tic()
hash_determine(pos_reviews[0:300],m,p_list[0:3],B)
print("Bloom filter: %.2f ms" % toc())
print('内存占用为')
print(tracemalloc.get_traced_memory())
tracemalloc.stop()
#print(B[0:1000])
#list_judge_1 = hash_determine(pos_reviews,m,p_list[0:k],B)
#list_positive = accurate_determine(pos_reviews,word_list )
#print(list_judge_1)
#print(list_positive)


Linear Search: 363.75 ms
Bloom filter: 10.29 ms
内存占用为
(19096893, 19107543)


#### hash_1函数输入一个字符串，一个质数，一个较大的数m，输出该字符串的哈希值（小于m）。hash_determine函数输入一个字符串列表compare，较大的数m，质数序列p_list, 映射完成的比特流B，输出一个列表，其元素是1或0，用于判断compare中的各个字符串是否在wordlist中。
#### 通过实验结果可以发现，暴力搜索需要的时间更多，符合我们的预期。

### 3. Use different bit array length ‘m’ and number of hash functions ‘k’ to implement the bloom filter algorithm. Then compare the impact of different m and k on the false positive rate.

In [ ]:
tracemalloc.start()
for mul in [3,4,5,6,7,8,9,10,11,12,13]:
    if mul == 0:
        continue
    m = mul*len(word_list)
    for k in range(5):
        if k == 0:
            continue
        B = np.zeros(m)
        for word in word_list:
            for p in p_list[0:k]:
                hash_value = hash_1(word,m,p)
                B[hash_value] = 1
        list_judge_1 = hash_determine(pos_reviews,m,p_list[0:k],B)
        list_judge_2 = hash_determine(neg_reviews,m,p_list[0:k],B)
        list_judge_1 = list_judge_1+list_judge_2
        list_positive = accurate_determine(pos_reviews,word_list )
        list_negative = accurate_determine(neg_reviews,word_list)
        list_positive = list_positive+list_negative
        fp = 0
        true_n = 0
        for i in range(len(list_judge_1)):
            if list_positive[i] == 0:
                true_n+=1
                if list_judge_1[i] == 1:
                    fp +=1
        fp_rate = fp/true_n
        print('m的值是'+str(m)+'哈希数量位list_judge_1k为'+str(k)+'假阳率为'+str(fp_rate))
print('内存占用为')
print(tracemalloc.get_traced_memory())

m的值是710208哈希数量位list_judge_1k为1假阳率为0.11351407518511658
m的值是710208哈希数量位list_judge_1k为2假阳率为0.2826444451472123
m的值是710208哈希数量位list_judge_1k为3假阳率为0.3006568871500404
m的值是710208哈希数量位list_judge_1k为4假阳率为0.5624833406370993
m的值是946944哈希数量位list_judge_1k为1假阳率为0.25292640062887684
m的值是946944哈希数量位list_judge_1k为2假阳率为0.3247570556637316
m的值是946944哈希数量位list_judge_1k为3假阳率为0.3353264331569888
m的值是946944哈希数量位list_judge_1k为4假阳率为0.4204913552024649
m的值是1183680哈希数量位list_judge_1k为1假阳率为0.2280943495958834
m的值是1183680哈希数量位list_judge_1k为2假阳率为0.30155141022918763
m的值是1183680哈希数量位list_judge_1k为3假阳率为0.3384301571740299
m的值是1183680哈希数量位list_judge_1k为4假阳率为0.3322001201733025
m的值是1420416哈希数量位list_judge_1k为1假阳率为0.06259402657366037
m的值是1420416哈希数量位list_judge_1k为2假阳率为0.037965276240473106
m的值是1420416哈希数量位list_judge_1k为3假阳率为0.03205600256610661
m的值是1420416哈希数量位list_judge_1k为4假阳率为0.05347937853234966
m的值是1657152哈希数量位list_judge_1k为1假阳率为0.06149620279470695
m的值是1657152哈希数量位list_judge_1k为2假阳率为0.17984883463521079
m的值是1657152哈希数量位list_judge

#### 可以从实验结果上看出，当m的值逐渐增加时，fp rate整体上会下降，符合我们的直觉。

## **Task3: Statistics Estimation**

Here we use the query stream (neg_reviews) from task 2 to estimate 1) the number of distinct words appeared, and 2)the surprise number of the stream.

### 1. 	Write a function that accurately counts the occurrence times of each word in neg_reviews.

In [74]:
from collections import Counter
tracemalloc.start()
negative_occurrence = Counter(neg_reviews)
print('内存占用为')
print(tracemalloc.get_traced_memory())
tracemalloc.stop()

内存占用为
(1321212, 1906101)


### 2. Implement the Flajolet-Martin alg. to estimate the number of distinct words occurred. Try multiple hash functions to improve the estimate.

In [77]:
print('真实不同单词的个数'+str(len(negative_occurrence)))
tracemalloc.start()
m = 20
list_size = []
list_max_num = []
p_list = [313,317,331,337,347,349,353,359,367,373,379,383,389,397,401,409,419,421,431, 433,439,443,449,457,461,463,467,479,487,491,499,503,509,521,523,541,547,557, 563,569,571,577,587,593,599,601,607,613,617,619,631,641,643,647,653,659,661, 673,677,683,691,701,709,719,727,733,739,743,751,757,761,769,773,787,797,809, 811,821,823,823,887,907,911,919,929,937, 941,947,953,967,971,977,983,991,997,1009,1013,1019,1021,1031,1033,1039,1049, 1051,1061,1063,1069,1087,1091,1093,1097,1103,1109,1117,1123,1129,1151,1153,62129,62131,62137,62141,62143,62171,62189,62191,62201,62207,62213,62219, 62233,62273,62297,62299,62303,62311,62323,62327,62347,62351,62383,62401, 62417,62423,62459,62467,62473,62477,62483,62497,62501,62507,62533,62539, 62549,62563,62581,62591,62597,62603,62617,62627,62633,62639,62653,62659, 62683,62687,62701,62723,62731,62743,62753,62761,62773,62791,62801,62819, 62827,62851,62861,62869,62873,62897,62903,62921,62927,62929,62939,62969, 62971,62981,62983,62987,62989,63029,63031,63059,63067,63073,63079,63097, 63103,63113,63127,63131,63149,63179,63197,63199,63211,63241,63247,63277, 63281,63299,63311,63313,63317,63331,63337,63347,63353,63361,63367,63377, 63389,63391,63397,63409,63419,63421,63439,63443,63463,63467,63473,63487,64123,64151,64153,64157,64171,64187,64189,64217,64223,64231,64237,64271, 64279,64283,64301,64303,64319,64327,64333,64373,64381,64399,64403,64433, 64439,64451,64453,64483,64489,64499,64513,64553,64567,64577,64579,64591, 64601,64609,64613,64621,64627,64633,64661,64663,64667,64679,64693,64709, 64717,64747,64763,64781,64783,64793,64811,64817,64849,64853,64871,64877, 64879,64891,64901,64919,64921,64927,64937,64951,64969,64997,65003,65011, 65027,65029,65033,65053,65063,65071,65089,65099,65101,65111,65119,65123, 65129,65141,65147,65167,65171,65173,65179,65183,65203,65213,65239,65257]
for k in range(20):
    p =  p_list[k]
    tail_num_zero = []
    for ng_review in neg_reviews:
        hash_value = hash_1(ng_review,2**m,p)
        tail_zero = len(bin(hash_value)) - len(bin(hash_value).rstrip('0'))
        tail_num_zero.append(tail_zero)
    max_num = max(tail_num_zero)
    list_max_num.append(max_num)
    mean_max = np.mean(list_max_num)
    list_size.append(2**max_num)
    print('k = '+str(k+1)+'时估计的不同单词的个数为'+str(2**mean_max))
print('内存占用为')
print(tracemalloc.get_traced_memory())
tracemalloc.stop()

真实不同单词的个数28480
k = 1时估计的不同单词的个数为8192.0
k = 2时估计的不同单词的个数为8192.0
k = 3时估计的不同单词的个数为8192.0
k = 4时估计的不同单词的个数为11585.237502960395
k = 5时估计的不同单词的个数为16384.0
k = 6时估计的不同单词的个数为14596.484598011326
k = 7时估计的不同单词的个数为13440.374472829133
k = 8时估计的不同单词的个数为17866.878691987582
k = 9时估计的不同单词的个数为20642.54648147761
k = 10时估计的不同单词的个数为24833.500225706357
k = 11时估计的不同单词的个数为25467.392520534748
k = 12时估计的不同单词的个数为29192.969196022652
k = 13时估计的不同单词的个数为27924.247230171546
k = 14时估计的不同单词的个数为28245.12540662098
k = 15时估计的不同单词的个数为29875.32250507566
k = 16时估计的不同单词的个数为28774.43539555614
k = 17时估计的不同单词的个数为31458.809834857868
k = 18时估计的不同单词的个数为31530.15068734735
k = 19时估计的不同单词的个数为30462.29131291547
k = 20时估计的不同单词的个数为28526.20085808741
内存占用为
(25954155, 25964890)


#### 用k个哈希估计出来的从右往左0的个数求平均，再来估计不同单词的个数

### 3.Estimate the surpise number with limited memory to store words.

In [78]:
import random
tracemalloc.start()
dic_ran = {}
cont_dic = {}
list_num = []
flag_dic = {}
for i in range(10):
    rand_num = random.uniform(0, len(neg_reviews))
    rand_num = int(rand_num)
    list_num.append(rand_num)
    dic_ran[neg_reviews[rand_num]] = rand_num
    cont_dic[neg_reviews[rand_num]] = 0
    flag_dic[neg_reviews[rand_num]] = 0

for t in range(len(neg_reviews)):
    word = neg_reviews[t]
    for j in range(len(list_num)):
        if list_num[j] == t :
            flag_dic[word] =1
    if word in dic_ran.keys():
        if flag_dic[word] == 1:
            cont_dic[word]+=1;
            
list_fx = []
for c in cont_dic.values():
    list_fx.append(len(neg_reviews)*(2*c-1))
print('Super number估计为'+str(np.mean(list_fx)))
print('内存占用为')
print(tracemalloc.get_traced_memory())
tracemalloc.stop()


Super number估计为2269844878.4
内存占用为
(164202, 174852)


#### 从neg_reviews中随机选择10个单词，用于计算Super number

[1, 2, 3, 4, 5, 9, 1, 34]
